## Baseball Data Prediction

My goal is to predict if a team will make the playoffs and win the World Series but looking at data prior to the midpoint in the season.  In the past, I've used an historical Bayes approach to this problem, and I would like to recreate it, and then try a few different approaches.

The first step is to read in the gamelog data that I pulled from Retrosheet: http://www.retrosheet.org/gamelogs/index.html

I've grabbed all seasons since 1980, which I think should be enough for what I'm working on.  A few of the challenges I might face using historical data: expansion teams, division changes, playoff format changes, rule changes, strike-shortened seasons, etc.


In [62]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import csv

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [63]:
path = "/home/craig/Documents/ML_Projects/Kaggle/Baseball/"
folder = "games/"
files = os.listdir(path + folder)
print(files)

['GL2007.TXT', 'GL1987.TXT', 'GL1981.TXT', 'GL1996.TXT', 'GL1985.TXT', 'GL1998.TXT', 'GL1993.TXT', 'GL2001.TXT', 'GL2011.TXT', 'GL2000.TXT', 'GL1980.TXT', 'GL2012.TXT', 'GL1997.TXT', 'GL2015.TXT', 'GL2016.TXT', 'GL2004.TXT', 'GL1989.TXT', 'GL2008.TXT', 'GL2003.TXT', 'GL1982.TXT', 'GL2005.TXT', 'GL2006.TXT', 'GL2014.TXT', 'GL1999.TXT', 'GL1991.TXT', 'GL1983.TXT', 'GL1986.TXT', 'GL2013.TXT', 'GL2002.TXT', 'GL1995.TXT', 'GL1990.TXT', 'GL2009.TXT', 'GL1984.TXT', 'GL1988.TXT', 'GL2010.TXT', 'GL1994.TXT', 'GL1992.TXT']


In [64]:
# Get the headers file and read in the game log files
with open(path + "gamelog_headers.csv", 'r') as f:
    reader = csv.reader(f)
    headers = list(reader)

headers = headers[0]
gamelog = pd.DataFrame()

for file in files:
    gamelog = gamelog.append(pd.read_csv(path + folder + file, header=None, names=headers))
    

In [65]:
gamelog.shape  # 83,229 rows, 161 columns

(83229, 161)

Now that the data has been read in, there is a ton of data cleaning to do.  First, I'm going to save the data in the sqlite database so I can get it back easily.

In [67]:
conn = sqlite3.connect("/home/craig/Documents/ML_Projects/Kaggle/Baseball/database.db")
try:
    gamelog.to_sql("gamelog",conn)
except ValueError:
    print("Table already exists.")

Table already exists.
